In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
PATH="../input/"

In [ ]:
df_train = pd.read_csv(f'{PATH}train.csv')
df_test = pd.read_csv(f'{PATH}test.csv')
df_train.head()

# Let's check some statistics about packages and hardware

In [ ]:
import fastai.utils.collect_env; fastai.utils.collect_env.show_install(1)

In [ ]:
import torch
print(torch.cuda.is_available())

# Import fastai packages

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
from fastai.imports import *
from fastai.tabular import * 

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

# Data Processing

In [ ]:
df_train.head()

In [ ]:
procs = [FillMissing, Normalize]
dep_var = 'target'

valid_idx = range(len(df_train)-2000, len(df_train))

In [ ]:
valid_idx

In [ ]:
df = df_train.drop('ID_code', axis=1)
#df_test = df_test.drop('ID_code', axis=1)

Use any path other than '../input', since in kaggle it is read-only path. tabular_learner will give error if we give input folder path

In [ ]:
data = TabularDataBunch.from_df('.', df, dep_var, valid_idx=valid_idx, procs=procs, test_df=df_test.drop('ID_code', axis=1))

In [ ]:
data.train_ds.cont_names

In [ ]:
#(cat_x,cont_x),y = next(iter(data.train_dl))
#for o in (cat_x, cont_x, y): print(to_np(o[:5]))

# Defining a model

In [ ]:
learn1 = tabular_learner(data, layers=[200,100], ps=[0.5, 0.2], metrics=[accuracy, metrics.roc_auc_score])
learn1.fit_one_cycle(3, 1e-2)

In [ ]:
learn2 = tabular_learner(data, layers=[200,100], ps=[0.5, 0.2], metrics=[accuracy, metrics.roc_curve])
learn2.fit(10, 1e-2)

In [ ]:
learn = tabular_learner(data, layers=[200,100], ps=[0.5, 0.2], metrics=accuracy)
learn.fit(10, 1e-3)

In [ ]:
#df_test.head()

In [ ]:
#??learn.predict

In [ ]:
#df.iloc[0]

In [ ]:
#learn.predict(df_test)

In [ ]:
#df_test.describe()

In [ ]:
test_preds, _ = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
result = to_np(test_preds[:,1])

In [ ]:
result

In [ ]:
learn.show_results()

In [ ]:
df_test['target'] = result

In [ ]:
df_test.head()

In [ ]:
output = df_test[['ID_code', 'target']]

In [ ]:
output.to_csv('submission.csv', index=False)